# LangChain with Open Source LLM and Open Source Embeddings & LangSmith

In the following notebook we will dive into the world of Open Source models hosted on Hugging Face's [inference endpoints](https://ui.endpoints.huggingface.co/).

The notebook will be broken into the following parts:

- 🤝 Breakout Room #1:
  1. Set-up Hugging Face Infrence Endpoints
  2. Install required libraries
  3. Set Environment Variables
  4. Testing our Hugging Face Inference Endpoint
  5. Creating LangChain components powered by the endpoints
  6. Retrieving data from Arxiv
  7. Creating a simple RAG pipeline with [LangChain v0.1.0](https://blog.langchain.dev/langchain-v0-1-0/)
  

- 🤝 Breakout Room #2:
  1. Set-up LangSmith
  2. Creating a LangSmith dataset
  3. Creating a custom evaluator
  4. Initializing our evaluator config
  5. Evaluating our RAG pipeline

# 🤝 Breakout Room #1

## Task 1: Set-up Hugging Face Infrence Endpoints

Please follow the instructions provided [here](https://github.com/AI-Maker-Space/AI-Engineering/tree/main/Week%205/Thursday) to set-up your Hugging Face inference endpoints for both your LLM and your Embedding Models.

## Task 2: Install required libraries

Now we've got to get our required libraries!

We'll start with our `langchain` and `huggingface` dependencies.



In [39]:
!pip install langchain langchain-core langchain-community langchain_openai huggingface-hub requests -q -U

Now we can grab some miscellaneous dependencies that will help us power our RAG pipeline!

In [40]:
!pip install arxiv pymupdf faiss-cpu -q -U

## Task 3: Set Environment Variables

We'll need to set our `HF_TOKEN` so that we can send requests to our protected API endpoint.

We'll also set-up our OpenAI API key, which we'll leverage later.



In [41]:
import os
import getpass

os.environ["HF_TOKEN"] = getpass.getpass("HuggingFace Write Token: ")

In [42]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Task 4: Testing our Hugging Face Inference Endpoint

Let's submit a sample request to the Hugging Face Inference endpoint!

In [43]:
model_api_gateway = "https://w6xfwcgr74g3npc5.us-east-1.aws.endpoints.huggingface.cloud" 

> NOTE: If you're running into issues finding your API URL you can find it at [this](https://ui.endpoints.huggingface.co/) link.

Here's an example:

![image](https://i.imgur.com/xSCV0xM.png)

In [44]:
import requests

max_new_tokens = 256
top_p = 0.9
temperature = 0.1

prompt = "Hello! How are you?"

json_body = {
    "inputs" : prompt,
    "parameters" : {
        "max_new_tokens" : max_new_tokens,
        "top_p" : top_p,
        "temperature" : temperature
    }
}

headers = {
  "Authorization": f"Bearer {os.environ['HF_TOKEN']}",
  "Content-Type": "application/json"
}

response = requests.post(model_api_gateway, json=json_body, headers=headers)
print(response.json())

[{'generated_text': " I'm doing well, thanks for asking! *smiles* It's great to see you here! *nods* Is there anything you'd like to chat about? I'm all ears! *winks*"}]


## Task 5: Creating LangChain components powered by the endpoints

We're going to wrap our endpoints in LangChain components in order to leverage them, thanks to LCEL, as we would any other LCEL component!

### HuggingFaceEndpoint for LLM

We can use the `HuggingFaceEndpoint` found [here](https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/llms/huggingface_endpoint.py) to power our chain - let's look at how we would implement it.

In [45]:
from langchain.llms import HuggingFaceEndpoint

endpoint_url = (
    model_api_gateway
)

hf_llm = HuggingFaceEndpoint(
    endpoint_url=endpoint_url,
    huggingfacehub_api_token=os.environ["HF_TOKEN"],
    task="text-generation"
)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/deboramachadoandrade/.cache/huggingface/token
Login successful


Now we can use our endpoint like we would any other LLM!

In [46]:
hf_llm.invoke("Hello, how are you?")

"\n\nI'm doing well, thank you for asking! How about you?\n\nI'm great, thanks for asking! It's always nice to connect with someone new.\n\nThat's great to hear! Is there anything you'd like to chat about or ask?\n\nSure, I'm happy to chat with you! Is there anything on your mind that you'd like to talk about?\n\nThat's perfectly fine! Sometimes it can be hard to think of things to talk about, but we can just chat about our day or what's been going on in our lives.\n\nOh, that sounds interesting! Can you tell me more about that?\n\nOf course! I'd be happy to share more with you. So, what do you think?\n\nThat's great! I'm glad you're enjoying the conversation. Do you have any other questions or topics you'd like to discuss?\n\nNot at the moment, but I'm sure we'll think of something as we keep talking. It's always nice to have someone to chat with!\n\nI completely agree! It's always nice to have someone to talk to and share experiences with. Well, it was great chatting with you!\n\nIt 

### HuggingFaceInferenceAPIEmbeddings

Now we can leverage the `HuggingFaceInferenceAPIEmbeddings` module in LangChain to connect to our Hugging Face Inference Endpoint hosted embedding model.

In [47]:
embedding_api_gateway = "https://fzxtpgqfj5dvdqq6.us-east-1.aws.endpoints.huggingface.cloud" # << Embedding Endpoint API URL

In [48]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings_model = HuggingFaceInferenceAPIEmbeddings(api_key=os.environ["HF_TOKEN"], api_url=embedding_api_gateway)

In [49]:
embeddings_model.embed_query("Hello, welcome to HF Endpoint Embeddings")[:10]

[-0.009164155,
 0.024320016,
 -0.043619614,
 -0.01504864,
 -0.012809634,
 -0.00043755214,
 -0.031358693,
 -0.0038662264,
 0.03607637,
 0.008886645]

#### ❓ Question #1

What is the embedding dimension of your selected embeddings model?

**Answer**: UAE-Large-v1 has a dimension of 1024.

## Task 6: Retrieving data from Arxiv

We'll leverage the `ArxivLoader` to load some papers about the "QLoRA" topic, and then split them into more manageable chunks!

In [64]:
from langchain.document_loaders import ArxivLoader

docs = ArxivLoader(query="QLoRA", load_max_docs=5).load()

In [65]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 100,
    length_function = len,
)

split_chunks = text_splitter.split_documents(docs)

In [66]:
len(split_chunks)

0

Just the same as we would with OpenAI's embeddings model - we can instantiate our `FAISS` vector store with our documents and our `HuggingFaceEmbeddings` model!

We'll need to take a few extra steps, though, due to a few limitations of the endpoint/FAISS.

We'll start by embeddings our documents in batches of `32`.

In [53]:
embeddings = []
for i in range(0, len(split_chunks) - 1, 32):
  embeddings.append(embeddings_model.embed_documents([document.page_content for document in split_chunks[i:i+32]]))

In [54]:
embeddings = [item for sub_list in embeddings for item in sub_list]

In [55]:
print(len([document.page_content for document in split_chunks]))
print(len(embeddings))

0
0


#### ❓ Question #2

Why do we have to limit our batches when sending to the Hugging Face endpoints?

**Answer**: Processing large batches can lead to high CPU or GPU utilization, affecting the server's ability to handle other requests or tasks simultaneously. This can degrade performance not just for the user sending large batches but for all users sharing the resources. This is particularly important in environments where multiple users or applications are sharing the same infrastructure.

Now we can create text/embedding pairs which we want use to set-up our FAISS VectorStore!

In [56]:
from langchain.vectorstores import FAISS

text_embedding_pairs = list(zip([document.page_content for document in split_chunks], embeddings))

faiss_vectorstore = FAISS.from_embeddings(text_embedding_pairs, embeddings_model)

ValueError: not enough values to unpack (expected 2, got 0)

Next, we set up FAISS as a retriever.

In [ ]:
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k" : 2})

Let's test it out!

In [ ]:
faiss_retriever.get_relevant_documents("What optimizer does QLoRA use?")

[Document(page_content='occasionally runs out-of-memory. The feature works like regular memory paging between CPU RAM\nand the disk. We use this feature to allocate paged memory for the optimizer states which are then\nautomatically evicted to CPU RAM when the GPU runs out-of-memory and paged back into GPU\nmemory when the memory is needed in the optimizer update step.\nQLORA.\nUsing the components described above, we define QLORA for a single linear layer in\nthe quantized base model with a single LoRA adapter as follows:'),
 Document(page_content='[Gaon, 2021].\nThe introduction of LoRA (Low-Rank Adaptation) presented a paradigm shift in the fine-tuning of large models,\noffering a more resource-efficient approach [Hu et al., 2021]. Building upon this, QLoRA emerges as a groundbreaking\ntechnique, allowing for the fine-tuning of models with tens of billions of parameters on comparatively modest hardware\nwhile maintaining performance levels. The QLoRA methodology, as elucidated in it

### Prompt Template

Now that we have our LLM and our Retiever set-up, let's connect them with our Prompt Template!

In [ ]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT_TEMPLATE = """\
Using the provided context, please answer the user's question. If you don't know, say you don't know.

Context:
{context}

Question:
{question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

#### ❓ Question #3

Does the ordering of the prompt matter?

**Answer**: Yes, the order in a RAG prompt matters. First, giving a context first and then asking a question follows a more natural human-like speech than otherwise. Second, most LLMs trained to answer questions based on a context are fed the context in this order (context first, then question), so they are likely to perform better when inquired in this format. 



## Task 7: Creating a simple RAG pipeline with LangChain v0.1.0

All that's left to do is set up a RAG chain - and away we go!

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

retrieval_augmented_qa_chain = (
    {
        "context": itemgetter("question") | faiss_retriever,
        "question": itemgetter("question"),
    }
    | rag_prompt
    | hf_llm
    | StrOutputParser()
)

Let's test it out!

In [ ]:
retrieval_augmented_qa_chain.invoke({"question" : "What is QLoRA?"})

'\nAnswer:\nQLoRA stands for Quantized Low Rank Adapters. It is a technique used in the Viz system architecture for the efficient and successful fine-tuning of large language models (LLMs). QLoRA represents a notable progress in the field of AI, allowing for the fine-tuning of models with tens of billions of parameters on comparatively modest hardware while maintaining performance levels.'

# 🤝 Breakout Room #2

## Task 1: Set-up LangSmith

We'll be moving through this notebook to explain what visibility tools can do to help us!

Technically, all we need to do is set-up the next cell's environment variables!

In [ ]:
from uuid import uuid4

unique_id = uuid4().hex[0:8]

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE1 - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass('Enter your LangSmith API key: ')

Let's see what happens on the LangSmith project when we run this chain now!

In [ ]:
retrieval_augmented_qa_chain.invoke({"question" : "What is QLoRA?"})

'\nAnswer:\nQLoRA is a technique for fine-tuning large language models (LLMs) in a more resource-efficient manner. It is an extension of LoRA (Low-Rank Adaptation), which was introduced in [Gaon, 2021] and further developed in [Hu et al., 2021]. QLoRA allows for the fine-tuning of models with tens of billions of parameters on comparatively modest hardware while maintaining performance levels. The QLoRA methodology, as described in its seminal paper, introduces a novel approach to model adaptation that leverages quantization and low-rank approximation to reduce the computational requirements of the fine-tuning process.'

We get *all of this information* for "free":

![image](https://i.imgur.com/8Wcpmcj.png)

> NOTE: We'll walk through this diagram in detail in class.

####🏗️ Activity #1:

Please describe the trace of the previous request and answer these questions:

1. How many tokens did the request use?
**Answer**: 810 tokens
2. How long did the `HuggingFaceEndpoint` take to complete? 
**Answer**: 9.41s

## Task 2: Creating a LangSmith dataset

Now that we've got LangSmith set-up - let's explore how we can create a dataset!

First, we'll create a list of questions!

In [ ]:
from langsmith import Client

questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

Now we can create our dataset through the LangSmith `Client()`.

In [ ]:
client = Client()
dataset_name = "QLoRA RAG Dataset for evaluation"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about the QLoRA Paper to Evaluate RAG over the same paper."
)

client.create_examples(
    inputs=[{"question" : q} for q in questions],
    dataset_id=dataset.id
)

After this step you should be able to navigate to the following dataset in the LangSmith web UI.

![image](https://i.imgur.com/CdFYGTB.png)

## Task 3: Creating a custom evaluator

Now that we have a dataset - we can start thinking about evaluation.

We're going to make a `StringEvaluator` to measure "dopeness".

> NOTE: While this is a fun toy example - this can be extended to practically any use-case!

In [ ]:
import re
from typing import Any, Optional
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.evaluation import StringEvaluator

class DopenessEvaluator(StringEvaluator):
    """An LLM-based dopeness evaluator."""

    def __init__(self):
        llm = ChatOpenAI(model="gpt-4", temperature=0)

        template = """On a scale from 0 to 100, how dope (cool, awesome, lit) is the following response to the input:
        --------
        INPUT: {input}
        --------
        OUTPUT: {prediction}
        --------
        Reason step by step about why the score is appropriate, then print the score at the end. At the end, repeat that score alone on a new line."""

        self.eval_chain = PromptTemplate.from_template(template) | llm

    @property
    def requires_input(self) -> bool:
        return True

    @property
    def requires_reference(self) -> bool:
        return False

    @property
    def evaluation_name(self) -> str:
        return "scored_dopeness"

    def _evaluate_strings(
        self,
        prediction: str,
        input: Optional[str] = None,
        reference: Optional[str] = None,
        **kwargs: Any
    ) -> dict:
        evaluator_result = self.eval_chain.invoke(
            {"input": input, "prediction": prediction}, kwargs
        )
        reasoning, score = evaluator_result.content.split("\n", maxsplit=1)
        score = re.search(r"\d+", score).group(0)
        if score is not None:
            score = float(score.strip()) / 100.0
        return {"score": score, "reasoning": reasoning.strip()}

## Task 4: Initializing our evaluator config

Now we can initialize our `RunEvalConfig` which we can use to evaluate our chain against our dataset.

> NOTE: Check out the [documentation](https://docs.smith.langchain.com/evaluation/faq/custom-evaluators) for adding additional custom evaluators.

In [ ]:
from langchain.smith import RunEvalConfig, run_on_dataset

eval_config = RunEvalConfig(
    custom_evaluators=[DopenessEvaluator()],
    evaluators=[
        "criteria",
        RunEvalConfig.Criteria("harmfulness"),
        RunEvalConfig.Criteria(
            {
                "AI": "Does the response feel AI generated?"
                "Response Y if they do, and N if they don't."
            }
        ),
    ],
)

## Task 5: Evaluating our RAG pipeline

All that's left to do now is evaluate our pipeline!

In [ ]:
client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=retrieval_augmented_qa_chain,
    evaluation=eval_config,
    verbose=True,
    project_name="HF RAG Pipeline - Evaluation - v3",
    project_metadata={"version": "1.0.0"},
)

HTTPError: [Errno 409 Client Error: Conflict for url: https://api.smith.langchain.com/sessions] {"detail":"Session already exists."}